In [1]:
"""
This is a skeleton of Assignment 1 for DS5110/CS5501 Spring 2024 (UVA).
NOTE you will need to change the .ipynb file name by following the naming convention.
Code should be commented well. 
Feel free to import any missing packages or add extra cells as you see fit. 
"""
import dask
from dask.distributed import Client
import dask.dataframe as dd
import time
import json
import numpy as np

In [2]:
# TODO: fill out the private IP address of your Dask scheduler and port here when creating a Dask client object
private_s = "172.31.75.157"
client = Client(f'{private_s}:8786')
client = client.restart()
print(client)

<Client: 'tcp://172.31.75.157:8786' processes=0 threads=0, memory=0 B>


In [3]:
%%time

# You should have already extracted the two .CSV files from stackoverflow.zip
# Load the .CSV files into the Dask dataframes
questions = dd.read_csv('/home/ubuntu/questions.csv', sample=2**30, assume_missing=True)
question_tags = dd.read_csv('/home/ubuntu/question_tags.csv', sample=2**30, assume_missing=True)

CPU times: user 575 ms, sys: 3.37 s, total: 3.94 s
Wall time: 13.2 s


**Task 1.** Get the percentage of missing values for all the columns in the questions table and the question_tags table.

In [4]:
%%time
# Task 1
# TODO: Start your implementation below this line
print(questions.isnull().mean().compute()*100)
print(question_tags.isnull().mean().compute()*100)

Id               0.000000
CreationDate     0.000000
ClosedDate      89.973578
DeletionDate    76.962907
Score            0.000000
OwnerUserId     23.920885
AnswerCount     10.114036
dtype: float64
Id     0.000000
Tag    0.027501
dtype: float64
CPU times: user 116 ms, sys: 0 ns, total: 116 ms
Wall time: 30.4 s


**Task 2.** Get mean, standard deviation, medium, min, and max of the Score column in the questions table.

In [5]:
%%time
# Task 2
# TODO: Start your implementation below this line
questions.Score.describe().compute()

CPU times: user 84.6 ms, sys: 8.39 ms, total: 93 ms
Wall time: 15 s


count    1.720382e+07
mean     1.204167e+00
std      1.564574e+01
min     -1.540000e+02
25%      0.000000e+00
50%      1.000000e+00
75%      4.000000e+00
max      1.690200e+04
Name: Score, dtype: float64

**Task 3.** Get the top 5 tags that have the highest number of questions (hint: manipulate the question_tags table).

In [6]:
%%time
# Task 3
# TODO: Start your implementation below this line
question_tags.Tag.value_counts().nlargest(5).compute()

CPU times: user 26.5 ms, sys: 0 ns, total: 26.5 ms
Wall time: 11.1 s


Tag
javascript    1649631
java          1563820
php           1365600
c#            1321027
android       1288338
Name: count, dtype: int64[pyarrow]

**Task 4.** Check if there are any dangling references to the question Id field from the questions table to question_tags table. Return 1 if there are dangling references; return 0 otherwise.

In [7]:
%%time
# Task 4
# TODO: Start your implementation below this line
dang = len(set(questions.Id.unique().compute())-set(question_tags.Id.unique().compute()))
if dang>0:
    print(1)
else:
    print(0)

0
CPU times: user 9.88 s, sys: 2.32 s, total: 12.2 s
Wall time: 58.5 s


**Task 5.** Create a new owner user table based on the questions table grouped by the OwnerUserId field. Refer to this link for Dask’s groupby operator. Refer to this link to see how to use Dask’s groupby.agg operator to apply multiple functions to different columns of the re-grouped dataframe. The new table should have the following fields: (1) OwnerUserId: the field that the table is grouped by; (2) AverageScore: aggregated by getting the average score value of the owner user; (3) NumQuestions: aggregated by getting count value of the Id field of corresponding owner users; (4) NumAnswers: aggregated by getting the sum value of the AnswerCount field of corresponding owner users. See the schema table with the corresponding aggregation functions as follows: (refer to class page)

In Task 5, you should output the top 5 owner users who asked the most number of questions.

In [8]:
%%time
# Task 5
# TODO: Start your implementation below this line
owner = questions.groupby("OwnerUserId")\
.agg({"Score":"mean", "Id":"count", "AnswerCount":"sum"})\
.rename(columns={"Score":"AverageScore", "Id":"NumQuestions", "AnswerCount":"NumAnswers"})
owner.NumQuestions.nlargest(5).compute()

CPU times: user 58.8 ms, sys: 10.8 ms, total: 69.6 ms
Wall time: 19.5 s


OwnerUserId
875317.0    2230
39677.0     2128
4653.0      1822
34537.0     1680
179736.0    1516
Name: NumQuestions, dtype: int64

**Task 6.** Create a new table by merging the questions table and the question_tags table using Id as the index. Then group the new table by Tag with the following aggregated fields: (1) Tag: the field that the table is grouped by; (2) AverageScore: aggregated by getting the average score value of the corresponding tags; (3) NumAnswers: aggregated by getting the sum value of the AnswerCount field of corresponding tags; (4) NumQuestions: aggregated by getting the count value of the Id field of corresponding tags; and (5) NumOwners: aggregated by getting the count value of the OwnerUserId field of corresponding tags. The schemas are listed as follows: (refer to class page)

This task should output the top 5 tags with the highest number of questions and the top 5 tags with the highest number of answers received.

In [9]:
%%time
# Task 6
# TODO: Start your implementation below this line
merged_df = questions.merge(question_tags, how="right", on="Id")
tag = merged_df.groupby("Tag")\
.agg({"Score": "mean", "AnswerCount":"sum", "Id":"count", "OwnerUserId":"count"})\
.rename(columns={"Score":"AverageScore", "AnswerCount":"NumAnswers", "Id":"NumQuestions", "OwnerId":"NumOwners"})
print(tag.NumQuestions.nlargest(5).compute())
print(tag.NumAnswers.nlargest(5).compute())

Tag
javascript    1649631
java          1563820
php           1365600
c#            1321027
android       1288338
Name: NumQuestions, dtype: int64
Tag
javascript    2252471.0
java          2194354.0
c#            1937822.0
php           1855546.0
jquery        1462029.0
Name: NumAnswers, dtype: float64
CPU times: user 250 ms, sys: 20.2 ms, total: 270 ms
Wall time: 1min 57s


**Task 7.** Kill the third and fourth Dask worker on vm3 by using ^C (Control+C). Dask scheduler should be able to transparently detect worker failures and will use the only two workers that are left for upcoming computations. Repeat Task 5 using only two Dask workers (running on vm2) and report the execution time of Task 7. Compare it with the execution time you saw from Task 5 and present your reasoning about why the execution time increases or decreases.

In [14]:
%%time
# Task 7: kill the third and fourth worker and repeat Task 5 with two workers 
# TODO: Start your implementation below this line
owner = questions.groupby("OwnerUserId")\
.agg({"Score":"mean", "Id":"count", "AnswerCount":"sum"})\
.rename(columns={"Score":"AverageScore", "Id":"NumQuestions", "AnswerCount":"NumAnswers"})
owner.NumQuestions.nlargest(5).compute()

CPU times: user 55.2 ms, sys: 0 ns, total: 55.2 ms
Wall time: 32.1 s


OwnerUserId
875317.0    2230
39677.0     2128
4653.0      1822
34537.0     1680
179736.0    1516
Name: NumQuestions, dtype: int64

In [11]:
# Add report for Task 7

**Answer 7** The time for question 5 with 4 workers took 19.5s while the time for question 5 with 2 workers took 32.1s. I think this is because the tasks of computing the mean, count, and sum could be distributed between the 4 workers so that it could be computed at the same time (more parallelism). But since the second time, there was only 2 workers, one of the workers needed to do two tasks, which increased the execution time.

**Task 8.** Kill the third and fourth Dask worker on vm3 by using ^C (Control+C). Dask scheduler should be able to transparently detect worker failures and will use the only two workers that are left for upcoming computations. Repeat Task 6 using only two Dask workers (running on vm2) and report the execution time of Task 8. Compare it with the execution time you saw from Task 6 and present your reasoning about why the execution time increases or decreases.

In [15]:
%%time
# Task 8: kill the third and fourth worker and repeat Task 6 with two workers 
# TODO: Start your implementation below this line
merged_df = questions.merge(question_tags, how="right", on="Id")
tag = merged_df.groupby("Tag")\
.agg({"Score": "mean", "AnswerCount":"sum", "Id":"count", "OwnerUserId":"count"})\
.rename(columns={"Score":"AverageScore", "AnswerCount":"NumAnswers", "Id":"NumQuestions", "OwnerId":"NumOwners"})
print(tag.NumQuestions.nlargest(5).compute())
print(tag.NumAnswers.nlargest(5).compute())

Tag
javascript    1649631
java          1563820
php           1365600
c#            1321027
android       1288338
Name: NumQuestions, dtype: int64
Tag
javascript    2252471.0
java          2194354.0
c#            1937822.0
php           1855546.0
jquery        1462029.0
Name: NumAnswers, dtype: float64
CPU times: user 312 ms, sys: 14.4 ms, total: 327 ms
Wall time: 4min 2s


In [13]:
# Add report for Task 8

**Answer 7** The time for question 6 with 4 workers took 1min 57s while the time for question 6 with 2 workers took 4min 2s. I believe this increase in execution time is due to the same reason mentioned above. The tasks of computing the mean, sum, and two counts were distributed between 2 workers instead of 4, which practically doubled the execution time.